In [2]:
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/afifai/pelatihan_machinelearning/master/data/train.csv', index_col=0)
df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Name      891 non-null    object 
 3   Sex       891 non-null    object 
 4   Age       714 non-null    float64
 5   SibSp     891 non-null    int64  
 6   Parch     891 non-null    int64  
 7   Ticket    891 non-null    object 
 8   Fare      891 non-null    float64
 9   Cabin     204 non-null    object 
 10  Embarked  889 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [4]:
df.drop(columns=['Name', 'Ticket', 'Cabin', 'Embarked'], inplace=True)

In [5]:
df.columns

Index(['Survived', 'Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [10]:
cat_col = ['Sex']
num_col = df.drop(cat_col, axis=1).columns.tolist()
num_col.remove('Survived')

In [12]:
cat_col, num_col

(['Sex'], ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare'])

In [13]:
from sklearn.model_selection import train_test_split
X = df.drop('Survived', axis=1)
y = df['Survived']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=46)

In [14]:
X_train_num = X_train[num_col]
X_test_num = X_test[num_col]

X_train_cat = X_train[cat_col]
X_test_cat = X_test[cat_col]

In [17]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')

In [18]:
X_train_num_impute = imputer.fit_transform(X_train_num)
X_test_num_impute = imputer.transform(X_test_num)

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_num_scaled = scaler.fit_transform(X_train_num_impute)
X_test_num_scaled = scaler.transform(X_test_num_impute)

In [21]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()

X_train_cat_ohe = ohe.fit_transform(X_train_cat).toarray()
X_test_cat_ohe = ohe.transform(X_test_cat).toarray()

In [25]:
import numpy as np
X_train_final = np.column_stack((X_train_num_scaled, X_train_cat_ohe))
X_test_final = np.column_stack((X_test_num_scaled, X_test_cat_ohe))

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X_train_final, y_train)
y_pred = knn.predict(X_test_final)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.91      0.88       104
           1       0.87      0.77      0.82        75

    accuracy                           0.85       179
   macro avg       0.86      0.84      0.85       179
weighted avg       0.86      0.85      0.85       179



In [27]:
new_data = X_test.iloc[0]

In [28]:
new_data_num = new_data[num_col]
new_data_cat = new_data[cat_col]

In [31]:
new_data_num = imputer.transform([new_data_num])
new_data_num = scaler.transform(new_data_num)

C:\Users\Judan Syamsul Hadad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but SimpleImputer was fitted with feature names
  warnings.warn(


In [32]:
new_data_cat = ohe.transform([new_data_cat]).toarray()

C:\Users\Judan Syamsul Hadad\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [33]:
new_data_final = np.column_stack((new_data_num, new_data_cat))

In [35]:
new_data_pred = knn.predict(new_data_final)
new_data_pred

array([0], dtype=int64)

In [36]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer

num_pipeline = make_pipeline(SimpleImputer(strategy='median'),
                                StandardScaler())

cat_pipeline = make_pipeline(OneHotEncoder())

data_pipeline = ColumnTransformer([
    ('num', num_pipeline, num_col),
    ('cat', cat_pipeline, cat_col)
    ])

final_pipeline = make_pipeline(data_pipeline, KNeighborsClassifier())

In [37]:
final_pipeline.fit(X_train,y_train)
y_pred = final_pipeline.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       104
           1       0.83      0.77      0.80        75

    accuracy                           0.84       179
   macro avg       0.84      0.83      0.83       179
weighted avg       0.84      0.84      0.84       179



In [38]:
import pickle
with open('pipeline.pkl') as f:
    pickle.dump(final_pipeline, f)

FileNotFoundError: [Errno 2] No such file or directory: 'pipeline.pkl'